In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm
import time
import urllib.request
import requests
import urllib
import re

In [7]:
station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
def input_keys():
    global stop_check
    key = input("s를 누르면 STOP : ")
    if key == ('s' or 'S'):
        stop_check = 1

# 검색 키워드를 받는 함수
def initkeyword(driver):
    print("더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.")
    driver.get("https://map.naver.com/v5/search/")
    while True:
        key = input("로그인이 완료됐으면 'Y'를 입력해주세요. :")
        if key == 'Y' or key == 'y':
            #print('검색 키워드 입력')
            #encText = urllib.parse.quote(f"{input('검색 키워드 : ')}")
            encText = urllib.parse.quote(station_list[0] + ' 음식점')
            driver.get("https://map.naver.com/v5/search/" + encText)
            break
        else:
            print('잘못된 입력. 다시 실행해주세요.')
            break
    return

# 맨 처음 스크롤을 끝까지 내려서 한 페이지 내 모든 음식점 HTML 태그 불러옴.
def init_scrollOnce(driver):
    driver.switch_to.default_content()
    next_iframe = driver.find_element(By.ID,"searchIframe")
    driver.switch_to.frame(next_iframe)

    for idx in tqdm(range(1,55)):
        try:
            ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
            driver.execute_script("arguments[0].scrollIntoView(true);",ele);
            driver.implicitly_wait(1)
        except:
            continue
    return 


# 가져올 장소
홍대입구역, 강남역, 명동역, 신촌역, 서울대입구역, 사당역, 혜화역, 가산디지털단지역, 서울역, 청담역, 종로3가역, 을지로입구역, 동대문역사문화공원역,   성신여대입구역, 이태원역, 성수역,  영등포역, 여의도역, 잠실역, 압구정역, 신사역
# 가져올 항목
이름, 역, 구, 카테고리, 방문자리뷰갯수, 블로그리뷰갯수, 평점, 주소, 영업시간, 부가서비스, 방문자리뷰글

In [8]:
#station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
station_list = ['성수역']

## 최종 코드

In [9]:
global stop_check
from threading import Thread
from selenium.common.exceptions import NoSuchElementException

shop_name_list = []
close_station_list = []
gu_list = []
category_list = []
visit_review_list = []
blog_review_list = []
rating_list = []
address_list = []
open_time_list = []
service_list = []
review_contents_list = []
detailed_review = []
latitude = []
longitude = []

driver = webdriver.Chrome(service=Service("../../EDA/driver/chromedriver"))

def main_action():
    
    
    global stop_check
    stop_check = 0
    
    for i in range(len(station_list)):
        # 다음 역 근처 음식점 검색
        driver.switch_to.default_content()
        input_tab = driver.find_element(By.CLASS_NAME, "input_search")
        input_clear  = driver.find_element(By.CLASS_NAME, 'btn_clear')
        input_clear.click()
        time.sleep(0.5)
        input_tab.send_keys(station_list[i] + " 음식점")
        input_tab.send_keys(Keys.RETURN)
        time.sleep(0.5)
        
        while stop_check != 1:    
            for idx in tqdm(range(1,55)):
                # 음식점 선택 프레임으로 iframe 이동
                driver.switch_to.default_content()
                next_iframe = driver.find_element(By.ID,"searchIframe")
                driver.switch_to.frame(next_iframe)
                
                # 음식점 링크 클릭 후 상세창 열기(li[idx]에서 idx 변경(1~53))
                time.sleep(1)
                try:
                    ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
                    driver.execute_script("arguments[0].scrollIntoView(true);",ele);
                except:
                    continue
                #ele.click()
                WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''')))
                time.sleep(0.5)
                try:
                    ele.click()
                except:
                    continue
                
                # 상세창으로 이동
                time.sleep(1)
                driver.implicitly_wait(3)
                driver.switch_to.default_content()
                time.sleep(1)
                next_iframe = driver.find_element(By.ID,"entryIframe")
                driver.switch_to.frame(next_iframe)
                driver.execute_script("window.scrollTo(0, 0)")
                driver.implicitly_wait(3)
                time.sleep(1)
                
                # 이름, 카테고리
                driver.execute_script("window.scrollTo(0, 0)")
                try:
                    shop_box = driver.find_element(By.CLASS_NAME, 'YouOG')
                    shop_name =  shop_box.find_element(By.CLASS_NAME, "Fc1rA").text
                    category = shop_box.find_element(By.CLASS_NAME, 'DJJvD').text
                except:
                    continue 
                #print(shop_name)
                #print(category)

                # 평점, 방문자리뷰갯수, 블로그리뷰갯수
                review_box = driver.find_elements(By.CLASS_NAME, 'PXMot')
                if len(review_box) > 2:
                    try:
                        rating = review_box[0].find_element(By.TAG_NAME, 'em').text
                        visit_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                        blog_review = review_box[2].find_element(By.TAG_NAME, 'em').text
                    except:
                        continue
                else:
                    rating = ''
                    try:
                        visit_review = review_box[0].find_element(By.TAG_NAME, 'em').text
                        blog_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                    except:
                        continue
                #print(rating)
                #print(visit_review)
                #print(blog_review)

                # 주소, 구, 역, 영업시간, 부가서비스
                try:
                    address = driver.find_element(By.CLASS_NAME, 'LDgIH').text
                    #print(address)
                    gu = address.split()[1]
                    #print(gu)
                except:
                    continue
                
                try:
                    station_box = driver.find_element(By.CLASS_NAME, 'nZapA')
                    close_station = station_box.find_element(By.TAG_NAME, 'strong').text
                except:
                    continue
                
                #print(close_station)
                try:
                    time_btn_box = driver.find_element(By.CLASS_NAME, 'y6tNq')
                    time_btn = time_btn_box.find_element(By.CLASS_NAME, '_UCia')
                    time_btn.click()
                except:
                    continue

                open_time = []
                time_box = driver.find_elements(By.CLASS_NAME, 'A_cdD')
                for i in range(1, len(time_box)):
                    if '-' in time_box[i].text or ':' in time_box[i].text:
                        open_time.append(time_box[i].text)
                open_time = str(open_time).replace('\\n', ' ').replace('[', '').replace(']', '').replace("'", '')
                #print(open_time)

                services = driver.find_elements(By.CLASS_NAME, 'vV_z_')
                service_words = []
                for service in services:
                    service = service.text 
                    if '단체석' in service:
                        service_words.append('단체석')
                    if '포장' in service:
                        service_words.append('포장')
                    if '예약' in service:
                        service_words.append('예약')
                    if '배달' in service:
                        service_words.append('포장')
                    if '무선 인터넷' in service:
                        service_words.append('무선 인터넷')
                    if '남/녀 화장실 구분' in service:
                        service_words.append('남/녀 화장실 구분')
                service_words = str(service_words).replace('[', '').replace(']', '').replace("'", "")
                #print(service_words)     
                
                # 리뷰 클릭
                review = driver.find_element(By.CLASS_NAME, 'place_fixed_maintab')
                i = 1
                while True:
                    review1 = review.find_element(By.XPATH, f'''//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]''')
                    if review1.text == "리뷰":
                        break
                    else: i+=1
                review1.click()
                #펼치기
                time.sleep(1)
                try:
                    spread = driver.find_element(By.CLASS_NAME, 'Tvx37')
                    while True:
                        try:
                            spread.click()
                        except:
                            break
                except:
                    continue
                time.sleep(0.3)
                review2 = driver.find_element(By.CLASS_NAME, 'uNsI9')
                # 자료 전처리
                reviews = review2.text
                reviews = reviews.split('\n')
                to_remove = ["이 키워드를 선택한 인원"]
                review_list = list(filter(lambda item: item not in to_remove, reviews))
                dic1 = review_list[::2]
                dic2 = review_list[1::2]
                review_dic = dict(zip(dic1, dic2))
                review = str(review_dic).replace("{", '').replace('}', '').replace('"', '').replace("'", '')
                #print(review)   
                
                shop_name_list.append(shop_name)
                close_station_list.append(close_station)
                gu_list.append(gu)
                category_list.append(category)
                visit_review_list.append(visit_review)
                blog_review_list.append(blog_review)
                rating_list.append(rating)
                address_list.append(address)
                open_time_list.append(open_time)
                service_list.append(service_words)
                review_contents_list.append(review)
               
                
                print(shop_name, close_station, gu, category, visit_review, blog_review, rating, address, open_time, service_words, review)
            
            
            # 음식점 선택 프레임으로 iframe 이동
            driver.switch_to.default_content()
            next_iframe = driver.find_element(By.ID,"searchIframe")
            driver.switch_to.frame(next_iframe)
                
            next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7)')))
            #next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div[4]/div[2]/a[7]''')))
            usable = next_page.get_attribute('aria-disabled')                                                                                              
            if usable == 'false':
                ActionChains(driver).move_to_element(next_page).perform()
                next_page.send_keys(Keys.ENTER)
                # next_page.click()
                driver.implicitly_wait(2)
            else:
                stop_check = 1
                print('Sucessfully Complete !!')
                break
            #except:
            #    break
                
            #usable = next_page.get_attribute('aria-disabled')                                                                                              
            
            
            # if usable == 'false':
            #     ActionChains(driver).move_to_element(next_page).perform()
            #     next_page.send_keys(Keys.ENTER)
            #     # next_page.click()
            #     driver.implicitly_wait(2)
            # else:
            #     print('Sucessfully Complete !!')
            #     break
    return 

from pynput import keyboard
def on_press(key):
    print('Key %s pressed' % key)

def on_release(key):
    global stop_check
    # print('Key %s released' %key)
    if key == keyboard.Key.esc: #esc 키가 입력되면 종료
        stop_check = 1
        return False

# def stop_func():
#     with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#         listener.join()        
        
import sys
if __name__ == '__main__':
    initkeyword(driver)
    driver.implicitly_wait(2)
    init_scrollOnce(driver)
    print('종료를 원하면 esc를 누르세요.')
    #t2 = Thread(target=main_action())
    main_action()

더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.


  0%|          | 0/54 [00:00<?, ?it/s]

종료를 원하면 esc를 누르세요.


  0%|          | 0/54 [00:00<?, ?it/s]

호보식당 성수직영점 성수역 성동구 육류,고기요리 1,459 862 4.45 서울 성동구 아차산로 97 1층 월 14:00 - 22:50 21:50 라스트오더, 화 14:00 - 22:50 21:50 라스트오더, 수 14:00 - 22:50 21:50 라스트오더, 목 14:00 - 22:50 21:50 라스트오더, 금 14:00 - 22:50 21:50 라스트오더, 토 13:00 - 21:50 20:50 라스트오더, 일 13:00 - 21:50 20:50 라스트오더, - 법정공휴일은 주말영업시간과 동일. (브레이크타임 없음) 단체석, 예약, 무선 인터넷, 남/녀 화장실 구분 음식이 맛있어요: 1112, 고기 질이 좋아요: 811, 친절해요: 567, 직접 잘 구워줘요: 421, 특별한 메뉴가 있어요: 382, 매장이 넓어요: 345, 단체모임 하기 좋아요: 246, 매장이 청결해요: 237, 인테리어가 멋져요: 226, 양이 많아요: 198, 가성비가 좋아요: 133, 재료가 신선해요: 72, 주차하기 편해요: 60, 특별한 날 가기 좋아요: 44, 뷰가 좋아요: 13, 메뉴 구성이 알차요: 10, 잡내가 적어요: 9, 혼밥하기 좋아요: 8, 반찬이 잘 나와요: 8, 아늑해요: 5, 건강한 맛이에요: 5, 화장실이 깨끗해요: 5, 차분한 분위기에요: 2, 좌석이 편해요: 2, 음식이 빨리 나와요: 2, 환기가 잘 돼요: 2, 비싼 만큼 가치있어요: 1, 룸이 잘 되어있어요: 1
토리아에즈 성수점 성수역 성동구 이자카야 347 212 4.64 서울 성동구 아차산로7길 7 1층 월 18:00 - 24:00, 화 18:00 - 24:00, 수 18:00 - 24:00, 목 18:00 - 24:00, 금 18:00 - 24:00, 토 17:00 - 24:00, 일 17:00 - 24:00 포장, 무선 인터넷, 예약 음식이 맛있어요: 207, 친절해요: 94, 가성비가 좋아요: 69, 특별한 메뉴가 있어요: 68, 기본 안주가 좋아요: 49, 술이 다양해요: 37, 대화

KeyboardInterrupt: 

In [24]:
print(len(shop_name_list), shop_name_list)
print(len(close_station_list), close_station_list)
print(len(gu_list), gu_list)
print(len(category_list), category_list)
print(len(visit_review_list), visit_review_list)
print(len(blog_review_list), blog_review_list)
print(len(rating_list), rating_list)
print(len(address_list), address_list)
print(len(open_time_list), open_time_list)
print(len(service_list), service_list)
print(len(review_contents_list), review_contents_list)

200 ['모르므로', '권식족발 성수본점', '고향양꼬치', '소문난성수감자탕', '대흥양다리바베큐 건대점', '피넛버터바나나', '매운향솥 마라탕 마라샹궈', '오마카세 오사이초밥 성수직영점', '호보식당 성수직영점', '성수족발', '소바식당', '봉자마라탕', '라화쿵부 건대점', '꿉당 성수점', '조조칼국수 성수점', '어니언 성수', '쎈느Scene', '앤드밀 성수점', '그리노 성수', '르프리크', '쿠나', '위풍당당족발', '문츠 바베큐', '이가네양꼬치 성수직영점', '37.5 시그니처 성수', '한계', '에이투비 Cafe&Bar', '텅 성수 스페이스', '라온석갈비', '봄의정원 성수점', '코끼리베이글 성수', '송계옥 성수점', '시옹마오', '갓잇 성수 홀리워터점', '마를리', '토리키치 성수점', '도도곱창 건대본점', '명봉양꼬치', '리틀포레스트', '대림국수 성수점', '담솥 성수점', '탐광', '로우키', '무근본', '이오로비스트로 성수점', '오로라베이커리카페 성수점', '아키야마', '투파인드피터 서울성수점', '가조쿠', '나누리잡화점', '다이닝 목로 성수점', '사위식당 성수점', '위풍당당족발', '권식족발 성수본점', '달구벌반점', '고우성수', '일일향 성수10호점', '헬렐레', '타치노미바이토라', '화화담 성수', '뚝 다이닝', '신라다랑원 성수점', '카페 포제', '이태리차차', '청주한씨', '피읻짜', '모르므로', '성수명당', '등불', '패티패티', '소바마에 니고', 'dxyz', '봇봇봇', '더 클래식 500 라구뜨', '호랑이분식 성수본점', '복만루', '성수갈비', '하루앤원데이', 'AMP', '오이노스 OINOS', '디플랫 성수', '훈춘양꼬치', '풀리김밥 성수점', '모리티아', '신사소곱창 성수점', '도죠', '칼', '이지더즈잇', '침묵을깨다', '안주나라', '이북집 찹쌀순대 성수점', '우오보 파스타 바', '파스트 팔레트', '로우커피스탠드 성수

In [25]:
import pandas as pd
df = pd.DataFrame({'이름': shop_name_list,
                   '역': close_station_list,
                   '구': gu_list,
                   '카테고리': category_list,
                   '방문자_리뷰_개수': visit_review_list,
                   '블로그_리뷰_개수': blog_review_list,
                   '평점': rating_list,
                   '주소': address_list,
                   '영업시간': open_time_list, 
                   '부가서비스': service_list,
                   '방문자_리뷰_글': review_contents_list
                   })
df

,이름,역,구,카테고리,방문자_리뷰_개수,블로그_리뷰_개수,평점,주소,영업시간,부가서비스,방문자_리뷰_글
0,모르므로,성수역,성동구,바(BAR),66,52,,서울 성동구 연무장길 17 2층,"일 17:00 - 24:00 23:00 라스트오더, 화 17:00 - 24:00 2...","단체석, 예약, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 56, 인테리어가 멋져요: 43, 친절해요: 37, 화장실이 깨끗..."
1,권식족발 성수본점,성수역,성동구,"족발,보쌈","1,356",508,4.69,서울 성동구 아차산로7길 4 1층,"일 13:00 - 22:30, 월 15:00 - 22:30, 화 12:00 - 22...","포장, 예약, 포장, 무선 인터넷","음식이 맛있어요: 831, 친절해요: 483, 양이 많아요: 348, 매장이 청결해..."
2,고향양꼬치,건대입구역,광진구,양꼬치,312,"3,495",4.38,서울 광진구 동일로18길 101,매일 11:30 - 01:00,"단체석, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 111, 친절해요: 50, 매장이 넓어요: 46, 단체모임 하기 ..."
3,소문난성수감자탕,성수역,성동구,감자탕,"10,242","4,755",4.39,서울 성동구 연무장길 45,매일 00:00 - 24:00,포장,"음식이 맛있어요: 4300, 양이 많아요: 1698, 가성비가 좋아요: 985, 매..."
4,대흥양다리바베큐 건대점,건대입구역,광진구,양꼬치,418,358,4.38,서울 광진구 아차산로30길 39 1층,매일 12:00 - 24:00,"단체석, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 202, 특별한 메뉴가 있어요: 99, 친절해요: 93, 양이 많..."
...,...,...,...,...,...,...,...,...,...,...,...
195,밥플러스 6호점,성수역,성동구,한식,686,32,4.11,서울 성동구 성수일로10길 26 하우스디세종타워 2층 202호,"월 11:00 - 14:00, 화 11:00 - 14:00, 수 11:00 - 14...","무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 129, 가성비가 좋아요: 76, 양이 많아요: 65, 매장이 넓..."
196,규카츠정 성수점,성수역,성동구,일식당,37,1,,서울 성동구 아차산로 96 지1층 101호,"매일 11:00 - 21:00 15:00 - 17:00 브레이크타임 14:30, 2...",무선 인터넷,"음식이 맛있어요: 31, 친절해요: 21, 고기 질이 좋아요: 15, 재료가 신선해..."
197,스탠드업플리즈바이턴온,성수역,성동구,"카페,디저트",228,365,4.65,서울 성동구 연무장3길 14,"일 09:00 - 18:00 17:30 라스트오더, 월 09:00 - 18:00 1...",,"커피가 맛있어요: 146, 친절해요: 68, 가성비가 좋아요: 57, 특별한 메뉴가..."
198,스시현,성수역,성동구,"초밥,롤",335,250,,"서울 성동구 성수일로10길 26 201호 스시현, 성수역 1번출구 도보 3분거리입니...","일 11:30 - 21:00, 월 11:30 - 22:00, 화 11:30 - 22...","단체석, 포장, 예약, 포장, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 137, 친절해요: 57, 재료가 신선해요: 56, 가성비가 좋아..."


In [26]:
df.to_csv('../data/성수역_기본.csv', encoding='utf-8')